### Paquetes necesarios

In [ ]:
import cv2  
import math 

from ultralytics import YOLO

### Entrenamiento del modelo de detección de matrículas

In [ ]:
model = YOLO('yolo11n.pt') #Contenedores
data_Yaml_path = "C:\\Users\\aleja\\OneDrive\\Documentos\\VC_Practicas_Alejandro_Rodriguez_Mesa\\Imagenes etiquetadas\\data.yaml"

results = model.train(data= data_Yaml_path, epochs=40, imgsz=640, batch=16, lr0=0.001, patience=5)

### Detección de matrícula en imagen estática. 

In [ ]:
from ultralytics import YOLO
import cv2
import math

# Nombres de las clases (en este caso, 'license_plate')
classNames = ['license_plate']  # Añade más nombres si tu modelo detecta más clases

# Cargar el modelo entrenado
model = YOLO('runs/detect/train2/weights/best.pt')

# Ruta de la imagen
image_path = "imagen_test1.jpeg"

# Cargar la imagen con OpenCV
img = cv2.imread(image_path)

# Verificar si la imagen se ha cargado correctamente
if img is None:
    print("Error al cargar la imagen. Verifica la ruta del archivo.")
else:
    # Aplicar el modelo a la imagen
    results = model.predict(source=image_path, imgsz=640)

    # Iterar sobre los resultados y obtener las coordenadas de la matrícula
    for result in results:
        boxes = result.boxes  # Obtener todas las cajas detectadas
        
        for box in boxes:
            # Coordenadas del contenedor
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)  # Convertir a valores enteros
            
            # Confianza de la predicción
            confidence = math.ceil((box.conf[0] * 100)) / 100
            print("Confianza --->", confidence)

            # Clase detectada (en este caso, siempre debería ser 'license_plate')
            cls = int(box.cls[0])
            label = f'{classNames[cls]} {confidence}'
            print("Clase -->", classNames[cls])

            # Conversión de identificador numérico de clase a un color RGB
            escala = int((cls / len(classNames)) * 255 * 3)
            if escala >= 255 * 2:
                R = 255
                G = 255
                B = escala - 255 * 2
            elif escala >= 255:
                R = 255
                G = escala - 255
                B = 0
            else:
                R = escala
                G = 0
                B = 0

            # Dibujar el rectángulo en la imagen
            cv2.rectangle(img, (x1, y1), (x2, y2), (R, G, B), 10)

            # Preparar el texto y las dimensiones del fondo
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 3
            font_thickness = 2  # Hacer el texto más grueso (bold)
            text_size = cv2.getTextSize(label, font, font_scale, font_thickness)[0]

            # Coordenadas del fondo (rectángulo oscuro detrás del texto)
            text_x, text_y = x1, y1 - 10
            text_w, text_h = text_size[0], text_size[1]

            # Fondo negro para el texto
            cv2.rectangle(img, (text_x, text_y - text_h - 5), (text_x + text_w, text_y + 5), (0, 0, 0), -1)

            # Poner el texto en blanco sobre el fondo negro
            cv2.putText(img, label, (text_x, text_y+3), font, font_scale, (255, 255, 255), font_thickness)

    # Guardar la imagen resultante con el rectángulo dibujado
    output_path = "result_img_test1.jpeg"
    cv2.imwrite(output_path, img)

    print(f"Imagen guardada correctamente en {output_path}")

### Deteccion de matrículas en video

In [ ]:
from ultralytics import YOLO
import cv2

# Carga del modelo YOLO entrenado (cambia el archivo si es necesario)
model = YOLO('runs/detect/train2/weights/best.pt')  # Aquí usas tu modelo entrenado

# Ruta del archivo de video que se quiere procesar
filename = "video_test1.mp4"

# Abrir el video
cap = cv2.VideoCapture(filename)

# Obtener las propiedades del video original
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Definir el codec y crear el VideoWriter para guardar el video procesado
output_path = "result_video_test.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (640, 640))

# Procesar el video con YOLO
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    resized_frame = cv2.resize(frame, (640, 640))
    

    # Aplicar el modelo a cada fotograma del video
    results = model(resized_frame)

    # Dibujar las detecciones sobre el fotograma
    annotated_frame = results[0].plot()  # Esto dibuja las detecciones sobre el frame

    # Mostrar el video en tiempo real con las detecciones (opcional)
    cv2.imshow('Video con detecciones', annotated_frame)

    # Guardar el fotograma procesado en el video de salida
    out.write(annotated_frame)

    # Presionar 'q' para salir manualmente del bucle
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado correctamente en {output_path}")

### Detectar matrículas, coches y personas.  

In [ ]:
from ultralytics import YOLO
import cv2

# Carga de ambos modelos
model_license_plate = YOLO('runs/detect/train2/weights/best.pt')  # Modelo entrenado para matrículas
model_yolo = YOLO('yolo11n.pt')  # Modelo YOLO preentrenado para coches y personas

# Ruta del archivo de video a procesar
filename = "video_test1.mp4"
cap = cv2.VideoCapture(filename)

# Obtener propiedades del video original
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Definir el codec y crear el VideoWriter para guardar el video procesado
output_path = "result_video_test.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (800, 800))

# Procesar el video con ambos modelos
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Redimensionar el fotograma para procesarlo con los modelos (opcional)
    resized_frame = cv2.resize(frame, (800, 800))

    # Aplicar el modelo de matrículas
    results_lp = model_license_plate(resized_frame)
    
    # Dibujar las detecciones de matrículas en el fotograma
    for box in results_lp[0].boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordenadas
        confidence = round(float(box.conf[0]) * 100) / 100  # Convertir a float antes de redondear
        label = f'license_plate {confidence}'
        
        # Dibujar rectángulo y etiqueta de la matrícula
        cv2.rectangle(resized_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Verde para matrículas
        cv2.putText(resized_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    # Aplicar el modelo YOLO preentrenado para coches y personas
    results_yolo = model_yolo(resized_frame)

    # Dibujar las detecciones de coches y personas en el fotograma
    for box in results_yolo[0].boxes:
        cls = int(box.cls[0])
        # Filtrar solo coches y personas
        if cls in [0, 2]:  # Clase 0: 'person', Clase 2: 'car'
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = round(float(box.conf[0]) * 100) / 100  # Convertir a float antes de redondear
            label = 'person' if cls == 0 else 'car'
            color = (255, 0, 0) if label == 'car' else (0, 0, 255)
            label = f'{label} {confidence}'

            # Dibujar rectángulo y etiqueta de coche o persona
            cv2.rectangle(resized_frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(resized_frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)

    # Mostrar el fotograma procesado en tiempo real
    cv2.imshow('Video con detecciones', resized_frame)

    # Guardar el fotograma procesado en el video de salida
    out.write(resized_frame)

    # Presionar 'q' para salir del bucle manualmente
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar los recursos
cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Video procesado guardado correctamente en {output_path}")


### Detección de coches, personas y matrículas en video

Teniendo en cuenta que las matrículas deben estar dentro de un coche, así se evitan falsos positivos. (El modelo puede detectar matrículas fuera de un coche, pero no se dibujan en el video)

In [ ]:
from ultralytics import YOLO
import cv2
def license_car_person_detector(video):
    # Carga de ambos modelos
    model_license_plate = YOLO('runs/detect/train2/weights/best.pt')  # Modelo entrenado para matrículas
    model_yolo = YOLO('yolo11n.pt')  # Modelo YOLO preentrenado para coches y personas

    # Ruta del archivo de video a procesar
    filename = video
    cap = cv2.VideoCapture(filename)

    # Procesar el video con ambos modelos
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Redimensionar el fotograma para procesarlo con los modelos (opcional)
        resized_frame = cv2.resize(frame, (640, 640))

        # Aplicar el modelo de matrículas
        results_lp = model_license_plate(resized_frame)
        license_boxes = []  # Lista para almacenar las detecciones de matrículas

        for box in results_lp[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Coordenadas
            confidence = round(float(box.conf[0]) * 100) / 100  # Convertir a float antes de redondear
            license_boxes.append((x1, y1, x2, y2, confidence))  # Guardar coordenadas y confianza de la matrícula

        # Aplicar el modelo YOLO preentrenado para coches y personas
        results_yolo = model_yolo(resized_frame)

        for box in results_yolo[0].boxes:
            cls = int(box.cls[0])
            if cls == 0:  # Clase 0: 'person'
                x1_person, y1_person, x2_person, y2_person = map(int, box.xyxy[0])
                confidence_person = round(float(box.conf[0]) * 100) / 100

                # Dibujar rectángulo y etiqueta de la persona
                cv2.rectangle(resized_frame, (x1_person, y1_person), (x2_person, y2_person), (0, 0, 255), 2)
                cv2.putText(resized_frame, f'person {confidence_person}', (x1_person, y1_person - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            if cls == 2:  # Clase 2: 'car'
                x1_car, y1_car, x2_car, y2_car = map(int, box.xyxy[0])
                confidence_car = round(float(box.conf[0]) * 100) / 100

                # Dibujar rectángulo y etiqueta del coche
                cv2.rectangle(resized_frame, (x1_car, y1_car), (x2_car, y2_car), (255, 0, 0), 2)
                cv2.putText(resized_frame, f'car {confidence_car}', (x1_car, y1_car - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                # Comprobar si alguna matrícula está dentro del coche
                for (x1_lp, y1_lp, x2_lp, y2_lp, confidence_lp) in license_boxes:
                    if x1_car <= x1_lp and y1_car <= y1_lp and x2_car >= x2_lp and y2_car >= y2_lp:
                        # Dibujar rectángulo y etiqueta de la matrícula dentro del coche
                        cv2.rectangle(resized_frame, (x1_lp, y1_lp), (x2_lp, y2_lp), (0, 255, 0), 2)
                        cv2.putText(resized_frame, f'license_plate {confidence_lp}', (x1_lp, y1_lp - 10), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        # Mostrar el fotograma procesado en tiempo real (opcional)
        cv2.imshow('Video con detecciones', resized_frame)

        # Presionar 'q' para salir del bucle manualmente
        if cv2.waitKey(20) & 0xFF in [ord('q'), 27]:
            break

    # Liberar los recursos
    cap.release()
    out.release()
    cv2.destroyAllWindows()



In [ ]:
license_car_person_detector("C0142.MP4")

In [ ]:
license_car_person_detector("video_test1.MP4")

### Se hace uso del OCR Tesseract para detectar textos en imagen estática

In [ ]:
from ultralytics import YOLO
import cv2
import math
import pytesseract
from pytesseract import Output

# Configurar la ruta de Tesseract si no está en el PATH
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Cargar el modelo YOLO entrenado
model = YOLO('runs/detect/train2/weights/best.pt')

# Ruta de la imagen
image_path = "imagen_test1.jpeg"
img = cv2.imread(image_path)

# Nombres de las clases
classNames = ['license_plate']

if img is None:
    print("Error al cargar la imagen. Verifica la ruta del archivo.")
else:
    results = model.predict(source=image_path, imgsz=640)

    for result in results:
        boxes = result.boxes
        
        for box in boxes:
            # Coordenadas de la caja delimitadora
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            
            if classNames[cls] == 'license_plate':  # Solo procesar si es una matrícula
                # Extraer la sub-imagen de la matrícula
                license_plate_img = img[y1:y2, x1:x2]
                
                # Convertir a RGB para Tesseract
                license_plate_img_rgb = cv2.cvtColor(license_plate_img, cv2.COLOR_BGR2RGB)
                
                # Aplicar Tesseract para extraer texto de la matrícula
                detected_text = pytesseract.image_to_string(license_plate_img_rgb, config='--psm 8') # PSM 8 es para texto en una sola línea
                
                # Mostrar el texto extraído
                print("Texto detectado en la matrícula:", detected_text)

                # Dibujar el rectángulo y el texto detectado en la imagen principal
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, detected_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Guardar la imagen con la matrícula detectada
    output_path = "result_img_with_text.jpeg"
    cv2.imwrite(output_path, img)
    print(f"Imagen guardada correctamente en {output_path}")


### Se hace uso del OCR easyocr para detectar textos en imagen estática

En mi caso tuve que utilizar la version 3.9.2 para hacer uso del OCR, con la versión 3.11 no funcionó.

In [ ]:
from ultralytics import YOLO
import cv2
import math
import easyocr

# Inicializar el lector de easyOCR
reader = easyocr.Reader(['en'])  # Puedes agregar otros idiomas si es necesario

# Cargar el modelo YOLO entrenado
model = YOLO('runs/detect/train2/weights/best.pt')

# Ruta de la imagen
image_path = "imagen_test1.jpeg"
img = cv2.imread(image_path)

# Nombres de las clases
classNames = ['license_plate']

if img is None:
    print("Error al cargar la imagen. Verifica la ruta del archivo.")
else:
    results = model.predict(source=image_path, imgsz=640)

    for result in results:
        boxes = result.boxes
        
        for box in boxes:
            # Coordenadas de la caja delimitadora
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = math.ceil((box.conf[0] * 100)) / 100
            cls = int(box.cls[0])
            
            if classNames[cls] == 'license_plate':  # Solo procesar si es una matrícula
                # Extraer la sub-imagen de la matrícula
                license_plate_img = img[y1:y2, x1:x2]
                
                # Usar easyOCR para extraer el texto
                detected_text = reader.readtext(license_plate_img, detail=0)  # 'detail=0' para obtener solo el texto
                detected_text = " ".join(detected_text) if detected_text else ""
                # Mostrar el texto extraído
                print("Texto detectado en la matrícula:", detected_text)

                # Dibujar el rectángulo y el texto detectado en la imagen principal
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(img, detected_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Guardar la imagen con la matrícula detectada
    output_path = "result_img_with_text_easyocr.jpeg"
    cv2.imwrite(output_path, img)
    print(f"Imagen guardada correctamente en {output_path}")


Los mejores resultados obtenidos fueron los del easyocr, con lo que usaré para el resto de la práctica, dicho OCR

## Easyocr para detección de texto de matrículas en video

In [ ]:
import csv
from ultralytics import YOLO
import cv2
import math
import easyocr
import re

# Inicializar el lector de easyOCR y cargar los modelos YOLO
reader = easyocr.Reader(['es', 'en'])
model = YOLO('runs/detect/train2/weights/best.pt')
model_yolo = YOLO('yolo11n.pt')  # Modelo YOLO preentrenado para coches y personas

# Ruta del video y configuración para guardar el video de salida
video_path = "video_test1.mp4"
cap = cv2.VideoCapture(video_path)
out = cv2.VideoWriter("video_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (int(cap.get(3)), int(cap.get(4))))


# Verificar apertura del video y preparar diccionario de conteo de matrículas
if not cap.isOpened():
    print("Error al abrir el video. Verifica la ruta del archivo.")
else:
    license_plate_counts = {}
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Procesar el frame con ambos modelos YOLO
        results_lp = model.predict(source=frame, imgsz=640, verbose=False)
        results_objs = model_yolo(source=frame, imgsz=640, verbose=False)

        # Detección de personas y coches
        license_boxes = []
        for box in results_objs[0].boxes:
            cls = int(box.cls[0])
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            confidence = round(float(box.conf[0]) * 100) / 100

            if cls == 0:  # Clase 0: 'person'
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, f'person {confidence}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            elif cls == 2:  # Clase 2: 'car'
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f'car {confidence}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)

                # Registrar el área del coche para buscar matrículas dentro
                license_boxes.append((x1, y1, x2, y2))

        # Detección de matrículas
        for result in results_lp:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = math.ceil((box.conf[0] * 100)) / 100
                cls = int(box.cls[0])

                # Procesar solo si la clase es 'license_plate'
                if cls == 0:
                    license_plate_img = frame[y1:y2, x1:x2]
                    detected_text_info = reader.readtext(license_plate_img, detail=1)
                    
                    # Extraer texto detectado y su confianza
                    if detected_text_info:
                        detected_text = detected_text_info[0][1].strip()
                        text_confidence = round(detected_text_info[0][2] * 100, 2)

                        # Verificar si la matricula está en un coche y formato de matrícula
                        for (car_x1, car_y1, car_x2, car_y2) in license_boxes:
                            if car_x1 <= x1 <= car_x2 and car_y1 <= y1 <= car_y2:
                                if len(re.findall(r'[A-Z]', detected_text)) >= 3 and len(re.findall(r'\d', detected_text)) >= 3 and len(re.findall(r'[^\w\s]', detected_text)) == 0:
                                    license_plate_counts[detected_text] = license_plate_counts.get(detected_text, 0) + 1
                                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                    cv2.putText(frame, f'{detected_text}  OCR_conf: {text_confidence}%', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Guardar el frame con las detecciones
        out.write(frame)
        frame_count += 1

    # Escribir los resultados acumulados en el archivo CSV
    with open("ocurrencias_detected_plates.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["License Plate", "Ocurrencias"])
        writer.writerows(license_plate_counts.items())

    # Liberar recursos
    cap.release()
    out.release()

print("Detección de matrículas completada y guardada en ocurrencias_detected_plates.csv y video_output.mp4")


In [ ]:
import csv
import math
import re
import cv2
from ultralytics import YOLO
import easyocr

# Inicializar el lector de easyOCR y los modelos YOLO
reader = easyocr.Reader(['es', 'en'])
plate_model = YOLO('runs/detect/train2/weights/best.pt')  # Modelo YOLO para matrículas
object_model = YOLO('yolo11n.pt')  # Modelo YOLO preentrenado para coches y personas

# Ruta del video y configuración para guardar el video de salida
video_path = "video_test1.mp4"
cap = cv2.VideoCapture(video_path)
out = cv2.VideoWriter("video_output.mp4", cv2.VideoWriter_fourcc(*'mp4v'), 30.0, (int(cap.get(3)), int(cap.get(4))))

# Parámetros de seguimiento y contadores
object_id = 0
tracked_objects = {}  # Almacena posición y contadores de pérdida de seguimiento por objeto
car_plate_counts = {}  # Almacena placas detectadas por cada ID de coche
MAX_DISTANCE = 50
MAX_LOST_FRAMES = 10

if not cap.isOpened():
    print("Error al abrir el video. Verifica la ruta del archivo.")
else:
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detección de coches y personas
        results_objs = object_model.predict(source=frame, imgsz=640, verbose=False)
        license_boxes = []

        for box in results_objs[0].boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cls = int(box.cls[0])
            confidence = round(float(box.conf[0]) * 100) / 100

            # Dibujar y etiquetar coches y personas en el marco
            if cls == 0:  # Clase 0: 'person'
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255), 2)
                cv2.putText(frame, f'person {confidence}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
            elif cls == 2:  # Clase 2: 'car'
                cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                cv2.putText(frame, f'car {confidence}', (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                license_boxes.append((x1, y1, x2, y2))  # Guardar áreas de coches para detección de matrículas

        # Detección y procesamiento de matrículas
        results_plates = plate_model.predict(source=frame, imgsz=640, verbose=False)
        current_objects = []

        for result in results_plates:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = math.ceil((box.conf[0] * 100)) / 100
                license_plate_img = frame[y1:y2, x1:x2]

                detected_text_info = reader.readtext(license_plate_img, detail=1)
                if detected_text_info:
                    detected_text = detected_text_info[0][1].strip()
                    letter_count = len(re.findall(r'[A-Z]', detected_text))
                    number_count = len(re.findall(r'\d', detected_text))
                    non_alphanumeric_count = len(re.findall(r'[^\w\s]', detected_text))

                    if letter_count >= 3 and number_count >= 3 and non_alphanumeric_count == 0:
                        matched_id = None
                        for obj_id, obj_data in tracked_objects.items():
                            prev_x, prev_y, _, _, lost_frames = obj_data
                            dist = math.sqrt((x1 - prev_x) ** 2 + (y1 - prev_y) ** 2)
                            if dist < MAX_DISTANCE:
                                matched_id = obj_id
                                tracked_objects[obj_id] = (x1, y1, x2, y2, 0)
                                break
                        
                        if matched_id is None:
                            object_id += 1
                            matched_id = object_id
                            tracked_objects[object_id] = (x1, y1, x2, y2, 0)
                            car_plate_counts[matched_id] = {}

                        car_plate_counts[matched_id][detected_text] = car_plate_counts[matched_id].get(detected_text, 0) + 1
                        current_objects.append(matched_id)

                        # Dibujar el rectángulo y el texto de la matrícula
                        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        cv2.putText(frame, f"{detected_text} ID:{matched_id}", (x1, y1 - 10),
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # Actualizar los objetos rastreados
        for obj_id in list(tracked_objects.keys()):
            if obj_id not in current_objects:
                x1, y1, x2, y2, lost_frames = tracked_objects[obj_id]
                if lost_frames >= MAX_LOST_FRAMES:
                    del tracked_objects[obj_id]
                else:
                    tracked_objects[obj_id] = (x1, y1, x2, y2, lost_frames + 1)

        # Guardar el frame con las detecciones en el archivo de salida
        out.write(frame)

    # Guardar los resultados de matrículas en el archivo CSV
    with open("detected_plates.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["License ID", "License Plate", "Occurrences"])
        for car_id, plates in car_plate_counts.items():
            for plate, count in plates.items():
                writer.writerow([car_id, plate, count])

    cap.release()
    out.release()

print("Detección de matrículas completada y guardada en detected_plates.csv y video_output.mp4")


### Anonimización de personas y matrículas 

In [ ]:
import csv
from ultralytics import YOLO
import cv2
import math
import easyocr
import re

# Inicializar el lector de easyOCR
reader = easyocr.Reader(['es', 'en'])

# Cargar los modelos YOLO entrenados
model = YOLO('runs/detect/train2/weights/best.pt')  # Para matrículas
model_yolo = YOLO('yolo11n.pt')  # Para coches y personas

# Ruta del video
video_path = "video_test1.mp4"
cap = cv2.VideoCapture(video_path)

# Configuración para guardar el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("video_output.mp4", fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Nombres de las clases para el modelo de matrículas
classNames = ['license_plate']  

# Parámetros de seguimiento
object_id = 0
tracked_objects = {}

# Diccionario para almacenar matrículas por identificador de coche
car_plate_counts = {}

# Parámetros de tolerancia de seguimiento
MAX_DISTANCE = 50
MAX_LOST_FRAMES = 15

if not cap.isOpened():
    print("Error al abrir el video. Verifica la ruta del archivo.")
else:
    frame_count = 0

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        # Detección de matrículas y personas en cada frame
        results_plates = model.track(source=frame, imgsz=640, verbose=False, show=False)
        results_people = model_yolo.track(source=frame, imgsz=640, verbose=False, show=False)
        
        current_objects = []
        
        # Procesar detección de matrículas
        for result in results_plates:
            boxes = result.boxes

            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                confidence = math.ceil((box.conf[0] * 100)) / 100
                cls = int(box.cls[0])

                if cls < len(classNames) and classNames[cls] == 'license_plate':
                    license_plate_img = frame[y1:y2, x1:x2]
                    detected_text = reader.readtext(license_plate_img, detail=0)
                    detected_text = detected_text[0].strip() if detected_text else ""

                    if detected_text:
                        letter_count = len(re.findall(r'[A-Z]', detected_text))
                        number_count = len(re.findall(r'\d', detected_text))
                        non_alphanumeric_count = len(re.findall(r'[^\w\s]', detected_text))

                        if letter_count >= 3 and number_count >= 3 and non_alphanumeric_count <= 0:
                            # Algoritmo de seguimiento
                            matched_id = None
                            for obj_id, obj_data in tracked_objects.items():
                                prev_x, prev_y, _, _, lost_frames = obj_data
                                dist = math.sqrt((x1 - prev_x) ** 2 + (y1 - prev_y) ** 2)
                                if dist < MAX_DISTANCE:
                                    matched_id = obj_id
                                    tracked_objects[obj_id] = (x1, y1, x2, y2, 0)
                                    break

                            if matched_id is None:
                                object_id += 1
                                matched_id = object_id
                                tracked_objects[object_id] = (x1, y1, x2, y2, 0)
                                car_plate_counts[object_id] = {}

                            # Actualiza el conteo de matrículas para el coche con matched_id
                            if detected_text in car_plate_counts[matched_id]:
                                car_plate_counts[matched_id][detected_text] += 1
                            else:
                                car_plate_counts[matched_id][detected_text] = 1

                            # Agregar a la lista de objetos actuales
                            current_objects.append(matched_id)

                            # Desenfoque en matrículas
                            roi = frame[y1:y2, x1:x2]
                            blurred_roi = cv2.GaussianBlur(roi, (51, 51), 0)
                            frame[y1:y2, x1:x2] = blurred_roi

        # Aplicar desenfoque en caras de personas detectadas
        for result in results_people:
            boxes = result.boxes

            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cls = int(box.cls[0])

                # Asumimos que la clase "0" en el modelo `model_yolo` corresponde a personas
                # Asumimos que la clase "0" en el modelo `model_yolo` corresponde a personas
                if cls == 0:  
                    # Para asegurarnos de que el desenfoque se aplica en la región correcta de la cara,
                    # intenta ajustar los límites y altura de la cara
                    face_y1 = y1  # Ajuste para no tomar todo el borde superior
                    face_y2 =  y1 + (y2 - y1) // 4   # Usar aproximadamente la mitad superior de la detección
                    face_roi = frame[face_y1:face_y2, x1:x2]

                    # Aplica desenfoque gaussiano en la región facial ajustada
                    blurred_face = cv2.GaussianBlur(face_roi, (51, 51), 0)
                    frame[face_y1:face_y2, x1:x2] = blurred_face

        # Aplicar desenfoque en matrículas de objetos rastreados que no se detectaron en el frame actual
        for obj_id, obj_data in list(tracked_objects.items()):  # Convertimos a una lista de items
            if obj_id not in current_objects:
                x1, y1, x2, y2, lost_frames = obj_data
                if lost_frames < MAX_LOST_FRAMES:
                    # Desenfocar usando la última ubicación conocida
                    roi = frame[y1:y2, x1:x2]
                    blurred_roi = cv2.GaussianBlur(roi, (99, 99), 0)
                    frame[y1:y2, x1:x2] = blurred_roi
                    # Incrementar los frames perdidos
                    tracked_objects[obj_id] = (x1, y1, x2, y2, lost_frames + 1)
                else:
                    del tracked_objects[obj_id]  # Eliminar si se pierde el objeto

        # Guardar el frame en el archivo de salida
        out.write(frame)
        frame_count += 1

    # Guardar resultados en el archivo CSV
    with open("detected_plates.csv", mode="w", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Car ID", "License Plate", "Occurrences"])
        for car_id, plates in car_plate_counts.items():
            for plate, count in plates.items():
                writer.writerow([car_id, plate, count])

    cap.release()
    out.release()

print("Detección de matrículas y desenfoque completados. Guardados en detected_plates.csv y video_output.mp4")


### CSV con todos los campos para las 3 clases + anonimización + función para mejorar la lógica del tracker

In [ ]:
import csv
from ultralytics import YOLO
import cv2
import math
import easyocr
import re

# Inicializar el lector de easyOCR
reader = easyocr.Reader(['es','en'])

# Cargar el modelo YOLO entrenado para matrículas y modelo para coches y personas
model = YOLO('runs/detect/train2/weights/best.pt')  # Modelo YOLO para detectar matrículas
model_yolo = YOLO('yolo11n.pt')  # Modelo YOLO para detectar coches y personas

# Ruta del video
video_path = "example.mp4"
cap = cv2.VideoCapture(video_path)

# Configuración para guardar el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter("video_output.mp4", fourcc, 30.0, (int(cap.get(3)), int(cap.get(4))))

# Nombres de las clases
classNames = ['license_plate']

# Parámetros de seguimiento
object_id_counter = 0
tracked_objects = {}

# Diccionario para almacenar matrículas por identificador de coche
car_plate_counts = {}
unique_cars = set()
unique_persons = set()
unique_plates = set()
# Parámetros de tolerancia de seguimiento
MAX_DISTANCE = 100
MAX_LOST_FRAMES = 20
DISTANCE_THRESHOLD = 10

# Función para encontrar el objeto más cercano en el diccionario de objetos rastreados
def find_closest_object(x1, y1, x2, y2):
    closest_id = None
    min_distance = float('inf')
    for obj_id, (tx1, ty1, tx2, ty2, _) in tracked_objects.items():
        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
        tracked_center_x, tracked_center_y = (tx1 + tx2) // 2, (ty1 + ty2) // 2
        distance = math.sqrt((center_x - tracked_center_x) ** 2 + (center_y - tracked_center_y) ** 2)
        if distance < min_distance and distance <= MAX_DISTANCE:
            min_distance = distance
            closest_id = obj_id
    return closest_id

# Abrir el archivo CSV para escritura
with open("detected_objects.csv", mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow([
        "fotograma", "tipo_objeto", "confianza", "identificador_tracking",
        "x1", "y1", "x2", "y2", "confianza_matrícula", "mx1", "my1", 
        "mx2", "my2", "texto_matricula"
    ])

    if not cap.isOpened():
        print("Error al abrir el video. Verifica la ruta del archivo.")
    else:
        frame_count = 0

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            results_yolo = model_yolo(frame)
            current_objects = []  # Para realizar seguimiento de objetos detectados en el cuadro actual
            license_boxes = []
            car_boxes = []

            # Detectar coches y personas
            for box in results_yolo[0].boxes:
                cls = int(box.cls[0])
                
                if cls == 2:  # Clase 2: 'car'
                    x1_car, y1_car, x2_car, y2_car = map(int, box.xyxy[0])
                    confidence_car = round(float(box.conf[0]) * 100) / 100
                    car_boxes.append((x1_car, y1_car, x2_car, y2_car))

                    closest_id = find_closest_object(x1_car, y1_car, x2_car, y2_car)
                    if closest_id is None:
                        closest_id = object_id_counter
                        object_id_counter += 1
                    
                    tracked_objects[closest_id] = (x1_car, y1_car, x2_car, y2_car, 0)
                    writer.writerow([
                        frame_count, "car", confidence_car, closest_id,
                        x1_car, y1_car, x2_car, y2_car, "", "", "", "", "", ""
                    ])
                    unique_cars.add(closest_id)
                    cv2.rectangle(frame, (x1_car, y1_car), (x2_car, y2_car), (255, 0, 0), 2)
                    cv2.putText(frame, f'car {confidence_car} ID: {closest_id}', (x1_car, y1_car - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 1)
                
                elif cls == 0:  # Clase 0: 'person'
                    # Código para manejar personas (igual al original)
                    x1_person, y1_person, x2_person, y2_person = map(int, box.xyxy[0])
                    confidence_person = round(float(box.conf[0]) * 100) / 100

                    closest_id = find_closest_object(x1_person, y1_person, x2_person, y2_person)
                    if closest_id is None:
                        closest_id = object_id_counter
                        object_id_counter += 1
                    
                    tracked_objects[closest_id] = (x1_person, y1_person, x2_person, y2_person, 0)
                    writer.writerow([
                        frame_count, "person", confidence_person, closest_id,
                        x1_person, y1_person, x2_person, y2_person, "", "", "", "", "", ""
                    ])
                    unique_persons.add(closest_id)
                    cv2.rectangle(frame, (x1_person, y1_person), (x2_person, y2_person), (0, 0, 255), 2)
                    cv2.putText(frame, f'person {confidence_person} ID: {closest_id}', (x1_person, y1_person - 10), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)
                    face_y1 = y1_person  # Ajuste para no tomar todo el borde superior
                    face_y2 =  y1_person + (y2_person - y1_person) // 4   # Usar aproximadamente la mitad superior de la detección
                    face_roi = frame[face_y1:face_y2, x1_person:x2_person]

                    # Aplica desenfoque gaussiano en la región facial ajustada
                    blurred_face = cv2.GaussianBlur(face_roi, (51, 51), 0)
                    frame[face_y1:face_y2, x1_person:x2_person] = blurred_face

            # Detectar matrículas y verificar si están dentro de un coche
            results = model.track(source=frame, imgsz=640, verbose=False, show=False)
            license_plate_detections = []

            for result in results:
                boxes = result.boxes

                for box in boxes:
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    confidence = math.ceil((box.conf[0] * 100)) / 100
                    cls = int(box.cls[0])

                    if classNames[cls] == 'license_plate':
                        # Verificar si la matrícula está dentro de algún coche detectado
                        inside_car = any(x1 >= cx1 and y1 >= cy1 and x2 <= cx2 and y2 <= cy2 
                                         for (cx1, cy1, cx2, cy2) in car_boxes)
                        
                        if inside_car:
                            license_plate_img = frame[y1:y2, x1:x2]
                            detected_text = reader.readtext(license_plate_img, detail=0)
                            detected_text = detected_text[0].strip() if detected_text else ""

                            if detected_text:
                                letter_count = len(re.findall(r'[A-Z]', detected_text))
                                number_count = len(re.findall(r'\d', detected_text))
                                non_alphanumeric_count = len(re.findall(r'[^\w\s]', detected_text))

                                if letter_count >= 3 and number_count >= 3 and non_alphanumeric_count <= 0:
                                    closest_id = find_closest_object(x1, y1, x2, y2)
                                    if closest_id is None:
                                        closest_id = object_id_counter
                                        object_id_counter += 1
                                    
                                    tracked_objects[closest_id] = (x1, y1, x2, y2, 0)
                                    writer.writerow([
                                        frame_count, "license_plate", confidence, closest_id,
                                        x1, y1, x2, y2, confidence, x1, y1, x2, y2, detected_text
                                    ])
                                    unique_plates.add(closest_id)
                                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                                    cv2.putText(frame, f'license_plate {confidence} ID: {closest_id}', (x1, y1 - 10), 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
                            # Desenfocar la matrícula detectada solo si está en un coche
                            roi = frame[y1:y2, x1:x2]
                            blurred_roi = cv2.GaussianBlur(roi, (51, 51), 0)
                            frame[y1:y2, x1:x2] = blurred_roi
                            current_objects.append(closest_id)

            # Incrementar el conteo de cuadros perdidos para los objetos no detectados en el cuadro actual
            for obj_id in list(tracked_objects.keys()):
                if obj_id not in current_objects:
                    x1, y1, x2, y2, lost_frames = tracked_objects[obj_id]
                    if lost_frames < MAX_LOST_FRAMES:
                        tracked_objects[obj_id] = (x1, y1, x2, y2, lost_frames + 1)
                    else:
                        del tracked_objects[obj_id]

            out.write(frame)
            frame_count += 1

    cap.release()
    out.release()

print("Detección completada y guardada en detected_objects.csv y video_output.mp4")
print(f"Número de coches detectados:{len(unique_cars)}. Número de personas: {len(unique_persons)}. Número de matrículas: {len(unique_plates)}.")


0: 384x640 4 cars, 1 bus, 10.0ms
Speed: 1.0ms preprocess, 10.0ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 bus, 9.1ms
Speed: 1.0ms preprocess, 9.1ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 5 cars, 1 bus, 10.5ms
Speed: 0.9ms preprocess, 10.5ms inference, 1.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 5 cars, 1 bus, 9.2ms
Speed: 1.6ms preprocess, 9.2ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 4 cars, 1 bus, 8.6ms
Speed: 2.0ms preprocess, 8.6ms inference, 3.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 persons, 3 cars, 1 bus, 8.7ms
Speed: 1.0ms preprocess, 8.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 cars, 1 bus, 10.2ms
Speed: 1.0ms preprocess, 10.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 2 cars, 1 bus, 7.7ms
Speed: 1.9ms preprocess, 7.